<a href="https://colab.research.google.com/github/soline013/Algorithm/blob/master/Implementation/Assignment%234/TSP_with_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
#For https://ichi.pro/ko/python-eul-sayonghan-tsp-traveling-salesman-problem-32409961217404
import numpy as np

In [ ]:
#For https://pro-gramming-diary.tistory.com/47
import math, random
import cv2

In [ ]:
#For https://gist.github.com/turbofart/3428880
import math
import random

In [ ]:
#For Assignment Refer.
import numpy as np
import csv

In [ ]:
#Mine.
import pandas as pd

# Assignment Refer.

In [ ]:
# given cities
cities = []
# solution
sol = []

# Euclidean distance measuring function
def distance(x, y):
  dist = np.linalg.norm(np.array(x) - np.array(y))
  return dist

# 1. get solution sequence and reorderiing (sort from 0)
with open('example_solution.csv', mode='r', newline='') as solution:

  # read solution sequence
  reader = csv.reader(solution)
  for row in reader:
    sol.append(int(row[0]))

  # reordering solution sequence
  idx = sol.index(0)

  front = sol[idx:]
  back = sol[0:idx]

  sol = front + back

  # expand 0 city (start) for simplicity
  sol.append(int(0))

# 2. get TSP city map
with open('TSP.csv', mode='r', newline='') as tsp:
  # read TSP city map
  reader = csv.reader(tsp)
  for row in reader:
    cities.append(row)

# 3. evaluate solution cost
total_cost = 0

for idx in range(len(sol)-1):

  # get city positions
  pos_city_1 = [float(cities[sol[idx]][0]), float(cities[sol[idx]][1])]
  pos_city_2 = [float(cities[sol[idx+1]][0]), float(cities[sol[idx+1]][1])]

  # distacne calculation
  dist = distance(pos_city_1, pos_city_2)

  # accmulation
  total_cost += dist

print('final cost: '+str(total_cost))

final cost: 51314.71550400719


# Mine.

In [ ]:
class City:
   def __init__(self, x=None, y=None):
      self.x = None
      self.y = None
      if x is not None:
         self.x = x
      else:
         self.x = int(random.random() * 200)
      if y is not None:
         self.y = y
      else:
         self.y = int(random.random() * 200)
   
   def getX(self):
      return self.x
   
   def getY(self):
      return self.y
   
   def distanceTo(self, city):
      #xDistance = abs(self.getX() - city.getX())
      #yDistance = abs(self.getY() - city.getY())
      #distance = math.sqrt( (xDistance*xDistance) + (yDistance*yDistance) )
      x = np.array([self.getX(), city.getX()])
      y = np.array([self.getY(), city.getY()])
      distance = np.linalg.norm(x - y)
      return distance
   
   def __repr__(self):
      return str(self.getX()) + ", " + str(self.getY())


class TourManager:
   destinationCities = []
   
   def addCity(self, city):
      self.destinationCities.append(city)
   
   def getCity(self, index):
      return self.destinationCities[index]
   
   def numberOfCities(self):
      return len(self.destinationCities)


class Tour:
   def __init__(self, tourmanager, tour=None):
      self.tourmanager = tourmanager
      self.tour = []
      self.fitness = 0.0
      self.distance = 0
      if tour is not None:
         self.tour = tour
      else:
         for i in range(0, self.tourmanager.numberOfCities()):
            self.tour.append(None)
   
   def __len__(self):
      return len(self.tour)
   
   def __getitem__(self, index):
      return self.tour[index]
   
   def __setitem__(self, key, value):
      self.tour[key] = value
   
   def __repr__(self):
      geneString = "|"
      for i in range(0, self.tourSize()):
         geneString += str(self.getCity(i)) + "|\n"
      return geneString
   
   def generateIndividual(self):
      for cityIndex in range(0, self.tourmanager.numberOfCities()):
         self.setCity(cityIndex, self.tourmanager.getCity(cityIndex))
      random.shuffle(self.tour)
   
   def getCity(self, tourPosition):
      return self.tour[tourPosition]
   
   def setCity(self, tourPosition, city):
      self.tour[tourPosition] = city
      self.fitness = 0.0
      self.distance = 0
   
   def getFitness(self):
      if self.fitness == 0:
         self.fitness = 1/float(self.getDistance())
      return self.fitness
   
   def getDistance(self):
      if self.distance == 0:
         tourDistance = 0
         for cityIndex in range(0, self.tourSize()):
            fromCity = self.getCity(cityIndex)
            destinationCity = None
            if cityIndex+1 < self.tourSize():
               destinationCity = self.getCity(cityIndex+1)
            else:
               destinationCity = self.getCity(0)
            tourDistance += fromCity.distanceTo(destinationCity)
         self.distance = tourDistance
      return self.distance
   
   def tourSize(self):
      return len(self.tour)
   
   def containsCity(self, city):
      return city in self.tour


class Population:
   def __init__(self, tourmanager, populationSize, initialise):
      self.tours = []
      for i in range(0, populationSize):
         self.tours.append(None)
      
      if initialise:
         for i in range(0, populationSize):
            newTour = Tour(tourmanager)
            newTour.generateIndividual()
            self.saveTour(i, newTour)
      
   def __setitem__(self, key, value):
      self.tours[key] = value
   
   def __getitem__(self, index):
      return self.tours[index]
   
   def saveTour(self, index, tour):
      self.tours[index] = tour
   
   def getTour(self, index):
      return self.tours[index]
   
   def getFittest(self):
      fittest = self.tours[0]
      for i in range(0, self.populationSize()):
         if fittest.getFitness() <= self.getTour(i).getFitness():
            fittest = self.getTour(i)
      return fittest
   
   def populationSize(self):
      return len(self.tours)

#유전 알고리즘 클래스.
class GA:
   def __init__(self, tourmanager):
      self.tourmanager = tourmanager
      self.mutationRate = 0.015
      self.tournamentSize = 5
      self.elitism = True
   
   def evolvePopulation(self, pop):
      newPopulation = Population(self.tourmanager, pop.populationSize(), False)
      elitismOffset = 0
      if self.elitism:
         newPopulation.saveTour(0, pop.getFittest())
         elitismOffset = 1
      
      for i in range(elitismOffset, newPopulation.populationSize()):
         parent1 = self.tournamentSelection(pop)
         parent2 = self.tournamentSelection(pop)
         child = self.crossover(parent1, parent2)
         newPopulation.saveTour(i, child)
      
      for i in range(elitismOffset, newPopulation.populationSize()):
         self.mutate(newPopulation.getTour(i))
      
      return newPopulation
   
   def crossover(self, parent1, parent2):
      child = Tour(self.tourmanager)
      
      startPos = int(random.random() * parent1.tourSize())
      endPos = int(random.random() * parent1.tourSize())
      
      for i in range(0, child.tourSize()):
         if startPos < endPos and i > startPos and i < endPos:
            child.setCity(i, parent1.getCity(i))
         elif startPos > endPos:
            if not (i < startPos and i > endPos):
               child.setCity(i, parent1.getCity(i))
      
      for i in range(0, parent2.tourSize()):
         if not child.containsCity(parent2.getCity(i)):
            for ii in range(0, child.tourSize()):
               if child.getCity(ii) == None:
                  child.setCity(ii, parent2.getCity(i))
                  break
      
      return child
   
   def mutate(self, tour):
      for tourPos1 in range(0, tour.tourSize()):
         if random.random() < self.mutationRate:
            tourPos2 = int(tour.tourSize() * random.random())
            
            city1 = tour.getCity(tourPos1)
            city2 = tour.getCity(tourPos2)
            
            tour.setCity(tourPos2, city1)
            tour.setCity(tourPos1, city2)
   
   def tournamentSelection(self, pop):
      tournament = Population(self.tourmanager, self.tournamentSize, False)
      for i in range(0, self.tournamentSize):
         randomId = int(random.random() * pop.populationSize())
         tournament.saveTour(i, pop.getTour(randomId))
      fittest = tournament.getFittest()
      return fittest

#Main 형식.
if __name__ == '__main__':
   
   tourmanager = TourManager()
   
   # 2. get TSP city map
   with open('TSP.csv', mode='r', newline='') as tsp:
    # read TSP city map
    reader = csv.reader(tsp)
    for x, y in reader:
      tourmanager.addCity(City(float(x), float(y)))
    #도시 수 검증.
    #print(tourmanager.numberOfCities())
  
   cities = []
   with open('TSP.csv', mode='r', newline='') as tsp:
    reader2 = csv.reader(tsp)
    for row in reader2:
      cities.append(row)

In [ ]:
if __name__ == '__main__':

  # Initialize population
  pop = Population(tourmanager, 50, True);
  print("Initial distance: " + str(pop.getFittest().getDistance()))
   
  # Evolve population for 50 generations
  ga = GA(tourmanager)
  pop = ga.evolvePopulation(pop)
  for i in range(0, 100):
    pop = ga.evolvePopulation(pop)
   
  # Print final results
  print("Finished")
  print("Final distance: " + str(pop.getFittest().getDistance()))
  print("Solution:")
  print(pop.getFittest())

  #Evolve population for 50 generations이기 때문에, 49 Index가 마지막이다.
  #print(pop.populationSize())
  #print(pop.tours[49])

  #tourmanager의 Index는 들어간 순서대로 있다.
  #for i in range(tourmanager.numberOfCities()):
  #  print(tourmanager.getCity(i))

Initial distance: 52216.22105981545
Finished
Final distance: 51136.80149418537
Solution:
|4.458725390262808, 47.94786330623919|
52.33278417340765, 47.50050160323029|
43.238769700379486, 60.97843180563197|
5.750866934725618, 24.233695392452226|
48.62455518274498, 91.55819737861341|
56.80893400241372, 95.80302846106828|
93.86473924815053, 13.364529711952445|
9.346733753318205, 41.781868175532345|
33.570615579004716, 74.47880572821266|
77.92826964396436, 84.3467796620566|
26.917037906684584, 63.18985285345161|
15.84695716499257, 69.52293274274449|
5.6202372603943225, 19.72371222083199|
73.24212995339383, 72.56888790114502|
60.11073018745198, 47.021002783248456|
18.172804756411832, 6.9594939814418|
83.72599113489977, 42.27632900152323|
16.072425874762754, 67.20285240802001|
15.806699566130888, 41.73201864124996|
5.89816395819085, 39.00711669183472|
67.68928537828509, 41.98942516492813|
3.9391898790200996, 58.17869133391157|
31.33086074498571, 71.08286999942358|
98.10751964947, 32.014342487

In [ ]:
#정리: 따라서 Index를 활용하여 새로운 리스트를 만들면 example_solution.csv 파일을 만들 수 있을 것이다.
#이를 시도해보자.

#pop.getFittest() 결과로 변수 생성.
temp = pop.getFittest()

#Print various temp.
#print(temp)
#print(temp.tourSize())
#print(temp.tour[0])

#새로운 변수에 index 정보를 저장.
length = tourmanager.numberOfCities()

temp2 = []
for i in range(length):
  temp2.append(tourmanager.getCity(i))

index = []
for i in range(length):
    j = temp.getCity(i)
    print(j)
    index.append(temp2.index(j))

#Print index.
print(index)
print(len(index))
print(temp2[index[0]])
print(temp2.index(temp2[index[0]]))

4.458725390262808, 47.94786330623919
52.33278417340765, 47.50050160323029
43.238769700379486, 60.97843180563197
5.750866934725618, 24.233695392452226
48.62455518274498, 91.55819737861341
56.80893400241372, 95.80302846106828
93.86473924815053, 13.364529711952445
9.346733753318205, 41.781868175532345
33.570615579004716, 74.47880572821266
77.92826964396436, 84.3467796620566
26.917037906684584, 63.18985285345161
15.84695716499257, 69.52293274274449
5.6202372603943225, 19.72371222083199
73.24212995339383, 72.56888790114502
60.11073018745198, 47.021002783248456
18.172804756411832, 6.9594939814418
83.72599113489977, 42.27632900152323
16.072425874762754, 67.20285240802001
15.806699566130888, 41.73201864124996
5.89816395819085, 39.00711669183472
67.68928537828509, 41.98942516492813
3.9391898790200996, 58.17869133391157
31.33086074498571, 71.08286999942358
98.10751964947, 32.01434248706764
51.47321292770426, 9.937813198079581
35.55013491957344, 8.96820540761657
46.63264685469082, 30.455120316334

In [ ]:
#Make a csv 'example_solution.csv'.
df = pd.DataFrame(index)
df.to_csv('example_solution.csv', index=False)

In [ ]:
#example_solution.csv를 사용하여 변수에 저장.
sol = []

# 1. get solution sequence and reorderiing (sort from 0)
with open('example_solution.csv', mode='r', newline='') as solution:

  # read solution sequence
  reader = csv.reader(solution)
  for row in reader:
    sol.append(int(row[0]))

  # reordering solution sequence
  idx = sol.index(0)

  front = sol[idx:]
  back = sol[0:idx]

  sol = front + back

  # expand 0 city (start) for simplicity
  sol.append(int(0))

In [ ]:
# Euclidean distance measuring function
def distance(x, y):
  dist = np.linalg.norm(np.array(x) - np.array(y))
  return dist

In [ ]:
# 3. evaluate solution cost
total_cost = 0

for idx in range(len(sol)-1):

  # get city positions
  pos_city_1 = [float(cities[sol[idx]][0]), float(cities[sol[idx]][1])]
  pos_city_2 = [float(cities[sol[idx+1]][0]), float(cities[sol[idx+1]][1])]

  # distacne calculation
  dist = distance(pos_city_1, pos_city_2)

  # accmulation
  total_cost += dist

print('final cost: '+str(total_cost))

final cost: 50925.89968330889


# https://gist.github.com/turbofart/3428880

In [ ]:
"""
This Python code is based on Java code by Lee Jacobson found in an article
entitled "Applying a genetic algorithm to the travelling salesman problem"
that can be found at: http://goo.gl/cJEY1
"""

import math
import random


class City:
   def __init__(self, x=None, y=None):
      self.x = None
      self.y = None
      if x is not None:
         self.x = x
      else:
         self.x = int(random.random() * 200)
      if y is not None:
         self.y = y
      else:
         self.y = int(random.random() * 200)
   
   def getX(self):
      return self.x
   
   def getY(self):
      return self.y
   
   def distanceTo(self, city):
      xDistance = abs(self.getX() - city.getX())
      yDistance = abs(self.getY() - city.getY())
      distance = math.sqrt( (xDistance*xDistance) + (yDistance*yDistance) )
      return distance
   
   def __repr__(self):
      return str(self.getX()) + ", " + str(self.getY())


class TourManager:
   destinationCities = []
   
   def addCity(self, city):
      self.destinationCities.append(city)
   
   def getCity(self, index):
      return self.destinationCities[index]
   
   def numberOfCities(self):
      return len(self.destinationCities)


class Tour:
   def __init__(self, tourmanager, tour=None):
      self.tourmanager = tourmanager
      self.tour = []
      self.fitness = 0.0
      self.distance = 0
      if tour is not None:
         self.tour = tour
      else:
         for i in range(0, self.tourmanager.numberOfCities()):
            self.tour.append(None)
   
   def __len__(self):
      return len(self.tour)
   
   def __getitem__(self, index):
      return self.tour[index]
   
   def __setitem__(self, key, value):
      self.tour[key] = value
   
   def __repr__(self):
      geneString = "|"
      for i in range(0, self.tourSize()):
         geneString += str(self.getCity(i)) + "|"
      return geneString
   
   def generateIndividual(self):
      for cityIndex in range(0, self.tourmanager.numberOfCities()):
         self.setCity(cityIndex, self.tourmanager.getCity(cityIndex))
      random.shuffle(self.tour)
   
   def getCity(self, tourPosition):
      return self.tour[tourPosition]
   
   def setCity(self, tourPosition, city):
      self.tour[tourPosition] = city
      self.fitness = 0.0
      self.distance = 0
   
   def getFitness(self):
      if self.fitness == 0:
         self.fitness = 1/float(self.getDistance())
      return self.fitness
   
   def getDistance(self):
      if self.distance == 0:
         tourDistance = 0
         for cityIndex in range(0, self.tourSize()):
            fromCity = self.getCity(cityIndex)
            destinationCity = None
            if cityIndex+1 < self.tourSize():
               destinationCity = self.getCity(cityIndex+1)
            else:
               destinationCity = self.getCity(0)
            tourDistance += fromCity.distanceTo(destinationCity)
         self.distance = tourDistance
      return self.distance
   
   def tourSize(self):
      return len(self.tour)
   
   def containsCity(self, city):
      return city in self.tour


class Population:
   def __init__(self, tourmanager, populationSize, initialise):
      self.tours = []
      for i in range(0, populationSize):
         self.tours.append(None)
      
      if initialise:
         for i in range(0, populationSize):
            newTour = Tour(tourmanager)
            newTour.generateIndividual()
            self.saveTour(i, newTour)
      
   def __setitem__(self, key, value):
      self.tours[key] = value
   
   def __getitem__(self, index):
      return self.tours[index]
   
   def saveTour(self, index, tour):
      self.tours[index] = tour
   
   def getTour(self, index):
      return self.tours[index]
   
   def getFittest(self):
      fittest = self.tours[0]
      for i in range(0, self.populationSize()):
         if fittest.getFitness() <= self.getTour(i).getFitness():
            fittest = self.getTour(i)
      return fittest
   
   def populationSize(self):
      return len(self.tours)


class GA:
   def __init__(self, tourmanager):
      self.tourmanager = tourmanager
      self.mutationRate = 0.015
      self.tournamentSize = 5
      self.elitism = True
   
   def evolvePopulation(self, pop):
      newPopulation = Population(self.tourmanager, pop.populationSize(), False)
      elitismOffset = 0
      if self.elitism:
         newPopulation.saveTour(0, pop.getFittest())
         elitismOffset = 1
      
      for i in range(elitismOffset, newPopulation.populationSize()):
         parent1 = self.tournamentSelection(pop)
         parent2 = self.tournamentSelection(pop)
         child = self.crossover(parent1, parent2)
         newPopulation.saveTour(i, child)
      
      for i in range(elitismOffset, newPopulation.populationSize()):
         self.mutate(newPopulation.getTour(i))
      
      return newPopulation
   
   def crossover(self, parent1, parent2):
      child = Tour(self.tourmanager)
      
      startPos = int(random.random() * parent1.tourSize())
      endPos = int(random.random() * parent1.tourSize())
      
      for i in range(0, child.tourSize()):
         if startPos < endPos and i > startPos and i < endPos:
            child.setCity(i, parent1.getCity(i))
         elif startPos > endPos:
            if not (i < startPos and i > endPos):
               child.setCity(i, parent1.getCity(i))
      
      for i in range(0, parent2.tourSize()):
         if not child.containsCity(parent2.getCity(i)):
            for ii in range(0, child.tourSize()):
               if child.getCity(ii) == None:
                  child.setCity(ii, parent2.getCity(i))
                  break
      
      return child
   
   def mutate(self, tour):
      for tourPos1 in range(0, tour.tourSize()):
         if random.random() < self.mutationRate:
            tourPos2 = int(tour.tourSize() * random.random())
            
            city1 = tour.getCity(tourPos1)
            city2 = tour.getCity(tourPos2)
            
            tour.setCity(tourPos2, city1)
            tour.setCity(tourPos1, city2)
   
   def tournamentSelection(self, pop):
      tournament = Population(self.tourmanager, self.tournamentSize, False)
      for i in range(0, self.tournamentSize):
         randomId = int(random.random() * pop.populationSize())
         tournament.saveTour(i, pop.getTour(randomId))
      fittest = tournament.getFittest()
      return fittest



if __name__ == '__main__':
   
   tourmanager = TourManager()
   
   # Create and add our cities
   city = City(60, 200)
   tourmanager.addCity(city)
   city2 = City(180, 200)
   tourmanager.addCity(city2)
   city3 = City(80, 180)
   tourmanager.addCity(city3)
   city4 = City(140, 180)
   tourmanager.addCity(city4)
   city5 = City(20, 160)
   tourmanager.addCity(city5)
   city6 = City(100, 160)
   tourmanager.addCity(city6)
   city7 = City(200, 160)
   tourmanager.addCity(city7)
   city8 = City(140, 140)
   tourmanager.addCity(city8)
   city9 = City(40, 120)
   tourmanager.addCity(city9)
   city10 = City(100, 120)
   tourmanager.addCity(city10)
   city11 = City(180, 100)
   tourmanager.addCity(city11)
   city12 = City(60, 80)
   tourmanager.addCity(city12)
   city13 = City(120, 80)
   tourmanager.addCity(city13)
   city14 = City(180, 60)
   tourmanager.addCity(city14)
   city15 = City(20, 40)
   tourmanager.addCity(city15)
   city16 = City(100, 40)
   tourmanager.addCity(city16)
   city17 = City(200, 40)
   tourmanager.addCity(city17)
   city18 = City(20, 20)
   tourmanager.addCity(city18)
   city19 = City(60, 20)
   tourmanager.addCity(city19)
   city20 = City(160, 20)
   tourmanager.addCity(city20)
   
   # Initialize population
   pop = Population(tourmanager, 50, True);
   print("Initial distance: " + str(pop.getFittest().getDistance()))
   
   # Evolve population for 50 generations
   ga = GA(tourmanager)
   pop = ga.evolvePopulation(pop)
   for i in range(0, 100):
      pop = ga.evolvePopulation(pop)
   
   # Print final results
   print("Finished")
   print("Final distance: " + str(pop.getFittest().getDistance()))
   print("Solution:")
   print(pop.getFittest())

Initial distance: 1867.0926582558393
Finished
Final distance: 946.7873367872219
Solution:
|60, 80|
20, 40|
20, 20|
60, 20|
100, 40|
160, 20|
200, 40|
180, 60|
180, 100|
120, 80|
40, 120|
20, 160|
60, 200|
80, 180|
100, 160|
140, 180|
180, 200|
200, 160|
140, 140|
100, 120|

|60, 80|
20, 40|
20, 20|
60, 20|
100, 40|
160, 20|
200, 40|
180, 60|
180, 100|
120, 80|
40, 120|
20, 160|
60, 200|
80, 180|
100, 160|
140, 180|
180, 200|
200, 160|
140, 140|
100, 120|

60, 200
180, 200
80, 180
140, 180
20, 160
100, 160
200, 160
140, 140
40, 120
100, 120
180, 100
60, 80
120, 80
180, 60
20, 40
100, 40
200, 40
20, 20
60, 20
160, 20


# https://pro-gramming-diary.tistory.com/47

In [ ]:
#클릭한 지점 저장
savepoints=[]

#각 지점의 위치를 표시하는 클래스
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def distanceTo(self, point): #거리 계산
        return math.sqrt(abs(self.x - point.x)**2 + abs(self.y - point.y)**2 )

    def __str__(self):
        return "("+str(self.x) + ", " + str(self.y)+")"

#모든 지점의 정보를 저장하는 클래스
class PointInfo:
    Pointarr = []

    def addPoint(self, point):
        self.Pointarr.append(point)

    def getPoint(self, index):
        return self.Pointarr[index]

    def numberOfPoints(self):
        return len(self.Pointarr)

class Route:
    def __init__(self, pointinfo, route=None):
        self.pointinfo = pointinfo
        self.route = []
        self.value = 0.0
        self.distance = 0
        for i in range(0, self.pointinfo.numberOfPoints()):
            self.route.append(None)

    def __len__(self):
        return len(self.route)

    def __getitem__(self, index):
        return self.route[index]

    def __setitem__(self, key, value):
        self.route[key] = value
        
    def __str__(self):
        string = '출발 -> '
        for i in range(0, self.routeSize()):
            string += str(self.getPoint(i)) + ' -> '
        string += '도착'
        return string
    
    #route리스트를 무작위로 섞어서 생성 
    def generateIndividual(self):
        for i in range(0, self.pointinfo.numberOfPoints()):
            self.setPoint(i, self.pointinfo.getPoint(i))
        random.shuffle(self.route)

    def getPoint(self, routePosition):
        return self.route[routePosition]
    #초기화
    def setPoint(self, routePosition, point):
        self.route[routePosition] = point
        self.value = 0.0
        self.distance = 0
    #점수를 거리의 역수로 계산
    def getValue(self):
        if self.value == 0:
            self.value = 1/float(self.getDistance())
        return self.value
    #전체 경로의 길이 반환, 마지막 인덱스의 지점은 0번째 지점과 거리 계산
    def getDistance(self):
        totalDistance = 0
        for i in range(0, self.routeSize()):
            start = self.getPoint(i)
            if i+1 < self.routeSize():
                end = self.getPoint(i+1)
                totalDistance += start.distanceTo(end)
            else:
                end = self.getPoint(0)
                totalDistance += start.distanceTo(end)
        self.distance = totalDistance
        return self.distance

    def routeSize(self):
        return len(self.route)
    
    #경로가 해당 지점을 포함하는지 여부를 반환
    def containsPoint(self, point):
        return point in self.route

#한 세대에 존재하는 모든 경로들을 저장
class Population:
    def __init__(self, pointinfo, populationSize, isFirst):
        #경로들을 저장하는 리스트
        self.routes = []
        #전부다 None으로 초기화
        for i in range(0, populationSize): 
            self.routes.append(None)
        #처음 생성한 경우 랜덤으로 여러 경로들을 생성
        if isFirst:
            for i in range(0, populationSize):
                newRoute = Route(pointinfo)
                newRoute.generateIndividual()
                self.saveRoute(i, newRoute) #모든 경로들을 routes 리스트에 저장
        
    def setitem(self, key, value):
        self.routes[key] = value

    def getitem(self, index):
        return self.routes[index]

    def saveRoute(self, index, route):
        self.routes[index] = route

    def getRoute(self, index):
        return self.routes[index]

    #최적의 값을 가지는 경로 찾아서 리턴
    def getBest(self):
        best = self.routes[0]
        for i in range(0, self.populationSize()):
            if best.getValue() < self.getRoute(i).getValue():
                best = self.getRoute(i)
        return best

    def populationSize(self):
        return len(self.routes)


class GA:
    def __init__(self, pointinfo):
        self.pointinfo = pointinfo
        self.mutationRate = 0.05
        self.tournamentSize = 5
        self.elitism = True
    #진화 진행
    def evolvePopulation(self, pop):
        newPopulation = Population(self.pointinfo, pop.populationSize(), False)
        newPopulation.saveRoute(0, pop.getBest()) #0번 인덱스에 최고 경로 저장
        #랜덤으로 두 개 뽑아서 결합, 저장
        for i in range(1, newPopulation.populationSize()):
            parent1 = self.tournamentSelection(pop)
            parent2 = self.tournamentSelection(pop)
            child = self.crossover(parent1, parent2)
            newPopulation.saveRoute(i, child)
        
        #돌연변이
        for i in range(1, newPopulation.populationSize()):
            self.mutate(newPopulation.getRoute(i))
        
        return newPopulation
    #교차
    def crossover(self, parent1, parent2):
        #공백의 child 경로 생성
        child = Route(self.pointinfo)
        
        startPos = int(random.random() * parent1.routeSize())
        endPos = int(random.random() * parent1.routeSize())
        
        #랜덤으로 child에 일부에 parent1의 유전자를 채움
        for i in range(0, child.routeSize()):
            if startPos < endPos and i > startPos and i < endPos:
                child.setPoint(i, parent1.getPoint(i))
            elif startPos > endPos:
                if not (i < startPos and i > endPos):
                    child.setPoint(i, parent1.getPoint(i))
        #앞선 과정에서 채우지 않은 유전자에 대해 순서대로 parent2의 유전자를 채움
        for i in range(0, parent2.routeSize()):
            if not child.containsPoint(parent2.getPoint(i)):
                for ii in range(0, child.routeSize()):
                    if child.getPoint(ii) == None:
                        child.setPoint(ii, parent2.getPoint(i))
                        break

        return child
   
    #돌연변이 확률에 따라 돌연변이 일으킴 (서로의 위치 바꿈)
    def mutate(self, route):
        for i in range(0, route.routeSize()):
            if random.random() < self.mutationRate:
                j = int(route.routeSize() * random.random())
                
                point1 = route.getPoint(i)
                point2 = route.getPoint(j)
                
                route.setPoint(j, point1)
                route.setPoint(i, point2)

    #랜덤으로 뽑아서 그 중 가장 best를 골라서 리턴 (다양성 구현)
    def tournamentSelection(self, pop):
        tournament = Population(self.pointinfo, self.tournamentSize, False)
        for i in range(0, self.tournamentSize):
            rand = int(random.random() * pop.populationSize())
            tournament.saveRoute(i, pop.getRoute(rand))
        best = tournament.getBest()
        return best
    
#클릭하면 점 표시, 총 지점 개수 카운팅
def mouse_handler(event,x,y,flags,param):
  if event == cv2.EVENT_LBUTTONUP:
      global points
      points+=1
      deplicated=map1.copy()
      savepoints.append((x,y))
      pointinfo.addPoint(Point(x,y))
      for xx, yy in savepoints:
          cv2.circle(deplicated, center=(xx, yy), radius=6, color=(0, 0, 255), thickness=-1, lineType=cv2.LINE_AA)
      cv2.imshow('img', deplicated)

#지점의 개수, 한 세대의 비교 대상 수, 세대 수
points = 0
populationSize = 50
generations = 300

map1 = cv2.imread('matzip.png')

#지점의 정보를 저장하는 phoneinfo 생성
pointinfo = PointInfo()
map1 = cv2.resize(map1, dsize=(750, 525), interpolation=cv2.INTER_AREA)

cv2.waitKey(0)

#사진이 나타나고 마우스로 지점을 클릭
#cv2.imshow('img', map1)
cv2.imshow('img', map1)
cv2.setMouseCallback('img', mouse_handler)
cv2.waitKey(0)

# 처음 경로의 모임을 생성하고 처음 케이스의 거리를 출력
group = Population(pointinfo, populationSize, True)
print("처음 거리: " + str(group.getBest().getDistance()))

ga = GA(pointinfo)

#지정한 세대 수만큼 진화 진행
for i in range(generations):
    group = ga.evolvePopulation(group) #진화 완료
    best = group.getBest() #최적의 경로를 찾음

    map_result = map1.copy()
    
    #최적의 경로에 대해 선 끗기
    cv2.line(map_result,pt1=(best[0].x, best[0].y),pt2=(best[points-1].x, best[points-1].y),color=(0 ,127, 255),thickness=3,lineType=cv2.LINE_AA)
    for j in range(1, points): 
        cv2.line(map_result,pt1=(best[j-1].x, best[j-1].y),pt2=(best[j].x, best[j].y),color=(0 ,127, 255),thickness=3,lineType=cv2.LINE_AA)
    #화면에 과정 출력
    cv2.putText(map_result, org=(15, 25), text='Generation: %d / %d' % (i+1, generations), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=0, thickness=1, lineType=cv2.LINE_AA)
    cv2.putText(map_result, org=(15, 50), text='Distance: %f' % (best.getDistance()), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=0, thickness=1, lineType=cv2.LINE_AA)
    cv2.imshow('result', map_result)
    cv2.waitKey(1)

# 최종 결과 출력
print("최종 거리: " + str(group.getBest().getDistance()))
print("경로:")
print(group.getBest())

cv2.waitKey(0)

error: ignored

# https://ichi.pro/ko/python-eul-sayonghan-tsp-traveling-salesman-problem-32409961217404

In [ ]:
def do_crossover(s1, s2, m):
  s1, s2 = s1.copy(), s2.copy()
  c1 = s2.copy()
  for i in range(m, len(s1)): c1.remove(s1[i])
  for i in range(m, len(s1)): c1.append(s1[i])
  c2 = s1.copy()
  for i in range(m, len(s2)): c2.remove(s2[i])
  for i in range(m, len(s2)): c2.append(s2[i])	
  return (c1, c2)
		
def do_mutation(s, m, n):
  i, j = min(m, n), max(m, n)
  s1 = s.copy()
  while i < j:
    s1[i], s1[j] = s1[j], s1[i]
	  i += 1
	  j -= 1
  return s1
	
def compute_fitness(G, s):
  l = 0
  for i in range(len(s)-1):
    l += G[s[i]][s[i+1]]
	  l += G[s[len(s)-1]][s[0]]	
  return l
	
def get_elite(G, gen, k):
  gen = sorted(gen, key=lambda s: compute_fitness(G, s))
  return gen[:k]
		
def TSP_GA(G, k=20, ntrial = 200):
  n_p = k
  mutation_prob = 0.1
  gen = []
  path = list(range(len(G)))
  while len(gen) < n_p:
    path1 = path.copy()
	  np.random.shuffle(path1)
	  if not path1 in gen:
		  gen.append(path1)
	
  for trial in range(ntrial):
	  gen = get_elite(G, gen, k)
	  gen_costs = [(round(compute_fitness(G, s),3), s) \
                for s in gen]
	  next_gen = []
	  for i in range(len(gen)):
      for j in range(i+1, len(gen)):
        c1, c2 = do_crossover(gen[i], gen[j], \
                              np.random.randint(0, len(gen[i])))
	      next_gen.append(c1)
	      next_gen.append(c2)
	    if np.random.rand() < mutation_prob:
	      m = np.random.randint(0, len(gen[i]))
	      while True:
          n = np.random.randint(0, len(gen[i]))
		      if m != n:
		        break
	       c = do_mutation(gen[i], m, n)
	       next_gen.append(c)
	  gen = next_gen